# model fine-tune (pretrain-finetune)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics import r2_score
import itertools
from dataPreprocess import read_dataset

In [ ]:
sourceSessionIndex = 15
targetSessionIndex = 16

In [ ]:
folderPath = './data'
fileList = sorted([i for i in os.listdir(folderPath) if i.endswith('.pkl.zip')])

In [ ]:
sourceSession = fileList[sourceSessionIndex]
targetSession = fileList[targetSessionIndex]

In [ ]:
sourceDataset = read_dataset(os.path.join(folderPath, sourceSession), ['velocity_x', 'velocity_y'])
targetDataset = read_dataset(os.path.join(folderPath, targetSession), ['velocity_x', 'velocity_y'])

In [ ]:
from model_tf import simpleDecodeModel

model = simpleDecodeModel()
model.compile(optimizer='adam', loss='mse')

In [ ]:
TRAIN_COUNT = 5000

In [ ]:
test_timestamp = targetDataset['timestamp'][TRAIN_COUNT:]

In [ ]:
train_x = sourceDataset['m1'][:TRAIN_COUNT, -5:, :]
train_y = sourceDataset['movement'][:TRAIN_COUNT, -5:, :]

In [ ]:
# z-score
train_y_mean, train_y_std = np.mean(train_y, axis=0), np.std(train_y, axis=0)

In [ ]:
train_y = (train_y - train_y_mean) / train_y_std

In [ ]:
# train
model.fit(x=train_x, y=train_y, batch_size=128, epochs=100, shuffle=True, verbose=0)

In [ ]:
train_x = targetDataset['m1'][:TRAIN_COUNT, -5:, :]
train_y = targetDataset['movement'][:TRAIN_COUNT]

In [ ]:
train_y = (train_y - train_y_mean) / train_y_std

In [ ]:
# finetune
model.fit(x=train_x, y=train_y, batch_size=128, epochs=100, shuffle=True, verbose=0)

In [ ]:
test_x = targetDataset['m1'][TRAIN_COUNT:, -5:, :]
test_y = targetDataset['movement'][TRAIN_COUNT:]

In [ ]:
# pred
pred = model.predict(x=test_x)

In [ ]:
# 還原
pred = pred * train_y_std + train_y_mean

In [ ]:
# 計算R2
r2 = [r2_score(y_pred=pred[:, i], y_true=test_y[:, i]) for i in range(pred.shape[-1])]
'r2_x =', r2_x, 'r2_y =', r2_y

In [ ]:
# 輸出
result = pd.DataFrame(columns=[])
axis = ['x', 'y']
for n, a in enumerate(axis):
    result = result.append(pd.DataFrame({
        'timestamp': test_timestamp,
        'value': test_y[:, n],
        'type': 'groundtruth',   
        'movement': 'velocity',
        'axis': a,          
        'test_on': targetSessionIndex,
        'r2': r2[n]     
    }))
    result = result.append(pd.DataFrame({
        'timestamp': test_timestamp,
        'value': pred[:, n],
        'type': 'prediction',   
        'movement': 'velocity',
        'axis': a, 
        'test_on': targetSessionIndex,  
        'r2': r2[n]           
    }))
result.reset_index(inplace=True, drop=True)

result.to_csv('./results/finetune.csv', index=False, header=True)